In [ ]:
import requests
import pandas as pd
from datetime import datetime
from datetime import datetime
from tqdm import tqdm
import os

API_PATH = "http://archive.org/wayback/available"
TARGET_PATH = "./../../data/archived_snapshots_urls.csv"

In [ ]:
def write_df_to_file(df, file_path):
    if not os.path.isfile(file_path):
        df.to_csv(file_path, mode='w', index=False, header=True)
    else:
        df.to_csv(file_path, mode='a', index=False, header=False)

def get_times_to_retrieve():
    stamps_14 = [
        datetime(y, m, 1, 0, 0, 0).strftime("%Y%m%d%H%M%S")
        for y in range(2022, 2026)   # 2022..2025
        for m in (1, 4, 7, 10)       # every 3 months
    ]
    return stamps_14

In [ ]:
instances_df = pd.read_csv('./../../data/instances_eng.csv')
recorded_urls = pd.read_csv(TARGET_PATH)
recorded_urls = recorded_urls[recorded_urls.archived_snapshots != '{}']

In [ ]:
recorded_urls.shape

In [ ]:
for iname in tqdm(instances_df.name):
    for tstamp in get_times_to_retrieve():

        retrieved = recorded_urls[(recorded_urls.url == iname) & (recorded_urls.timestamp == int(tstamp))]
        if retrieved.shape[0]:
            if retrieved.archived_snapshots.iloc[0]:
                continue

        response = requests.get(API_PATH, params={"url": iname, "timestamp": tstamp})
        if response.status_code != 200:
             print(iname, response.status_code, response.reason)
             break
        urls_to_retrieve = pd.DataFrame([response.json()])
        write_df_to_file(urls_to_retrieve, TARGET_PATH)
        

In [ ]:
if False:
    """
    The following code can also be used to get snapshots in a given timeframe.
    """
    from waybackpy import WaybackMachineCDXServerAPI
    url = "https://mastodon.xyz/about"
    user_agent = "Mozilla/5.0 (Windows NT 5.1; rv:40.0) Gecko/20100101 Firefox/40.0"
    cdx = WaybackMachineCDXServerAPI(url, user_agent, start_timestamp=2016, end_timestamp=2017)
    for item in cdx.snapshots():
        print(item.archive_url)